# Surfaces: Test Functions for Optimization

A quick introduction to benchmarking optimization algorithms.

**What you'll learn:**
- What test functions are and why they matter
- How to use Surfaces in just a few lines of code
- How to visualize optimization landscapes
- Practical tips for benchmarking optimizers

**Prerequisites:**
- Python basics (functions, dictionaries, loops)
- NumPy fundamentals
- No prior knowledge of optimization required

**Time to complete:** ~15 minutes

## Setup

Install Surfaces with visualization support, then import the modules we'll use throughout this notebook.

In [ ]:
# Uncomment to install:
# !pip install surfaces[viz]

In [ ]:
# All imports for this notebook
import time

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from surfaces import presets
from surfaces.test_functions import (
    AckleyFunction,
    EggholderFunction,
    HimmelblausFunction,
    RastriginFunction,
    RosenbrockFunction,
    SphereFunction,
)
from surfaces.visualize import plot_contour, plot_surface

---

## Why Test Functions?

When developing optimization algorithms, you need reliable ways to test them. Real-world problems are often expensive to evaluate, hard to visualize, and lack a known optimal solution.

**Test functions** solve these problems:

| Property | Benefit |
|----------|--------|
| Known global optimum | Measure how close your optimizer gets |
| Fast evaluation | Run thousands of experiments quickly |
| Diverse characteristics | Test different algorithm weaknesses |
| Standardized | Compare results with published research |

Surfaces provides **50+ test functions** ready to use.

---

## 1. Your First Test Function

The **Sphere function** is the simplest test case: just the sum of squared values. The minimum is at the origin where all parameters equal zero.

In [ ]:
sphere = SphereFunction(n_dim=2)

# Functions are callable - just pass parameters
print(f"f(1, 2) = {sphere({'x0': 1.0, 'x1': 2.0})}")
print(f"f(0, 0) = {sphere({'x0': 0.0, 'x1': 0.0})}  <- global minimum")

### Flexible Input Formats

Surfaces accepts whatever format fits your workflow. All of these produce the same result:

In [ ]:
sphere = SphereFunction(n_dim=2)

print("Dictionary: ", sphere({"x0": 1.0, "x1": 2.0}))
print("NumPy array:", sphere(np.array([1.0, 2.0])))
print("Python list:", sphere([1.0, 2.0]))
print("Kwargs:     ", sphere(x0=1.0, x1=2.0))

---

## 2. Visualizing Landscapes

Understanding an optimization landscape visually reveals why some problems are harder than others. Surfaces includes built-in plotting with Plotly.

### The Sphere: A Simple Bowl

Smooth and convex. Any gradient-based optimizer finds the minimum easily.

In [ ]:
fig = plot_surface(SphereFunction(n_dim=2), resolution=60)
fig.show()

### The Rastrigin: Many Local Minima

Same global minimum at the origin, but surrounded by traps. Tests an optimizer's ability to escape local optima.

In [ ]:
fig = plot_surface(RastriginFunction(n_dim=2), resolution=80)
fig.show()

### The Ackley: Deceptive Plateau

A flat outer region with a deep hole at the center. Many optimizers get stuck on the plateau, never finding the global minimum.

In [ ]:
fig = plot_surface(AckleyFunction(), resolution=80)
fig.show()

### Contour Plots

2D contour plots are often easier to read. The **Rosenbrock function** (the "banana valley") has its minimum at (1, 1), but the narrow curved valley makes optimization difficult.

In [ ]:
fig = plot_contour(RosenbrockFunction(n_dim=2), resolution=100)
fig.show()

---

## 3. Function Gallery

Different functions test different optimizer capabilities. Here's a side-by-side comparison of four common test functions.

In [ ]:
functions = [
    (SphereFunction(n_dim=2), "Sphere (Easy)"),
    (RastriginFunction(n_dim=2), "Rastrigin (Many Traps)"),
    (AckleyFunction(), "Ackley (Deceptive)"),
    (HimmelblausFunction(), "Himmelblau (4 Minima)"),
]

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{"type": "surface"}] * 2] * 2,
    subplot_titles=[name for _, name in functions],
)

In [ ]:
for idx, (func, _) in enumerate(functions):
    row, col = idx // 2 + 1, idx % 2 + 1
    space = func.search_space
    names = sorted(space.keys())
    x, y = space[names[0]], space[names[1]]

    z = np.array([[func({names[0]: xi, names[1]: yi}) for xi in x] for yi in y])
    func.reset()

    fig.add_trace(
        go.Surface(x=x, y=y, z=z, colorscale="Viridis", showscale=False), row=row, col=col
    )

fig.update_layout(height=700, width=800, title_text="Test Function Gallery")
fig.show()

---

## 4. Automatic Data Collection

One of Surfaces' most useful features: every evaluation is automatically tracked. No manual bookkeeping needed.

In [ ]:
func = RastriginFunction(n_dim=2)

# Simulate random search
np.random.seed(42)
for _ in range(100):
    func({"x0": np.random.uniform(-5, 5), "x1": np.random.uniform(-5, 5)})

In [ ]:
print(f"Evaluations:  {func.n_evaluations}")
print(f"Best score:   {func.best_score:.4f}")
print(f"Best params:  x0={func.best_params['x0']:.3f}, x1={func.best_params['x1']:.3f}")
print(f"True optimum: {func.f_global}")

### Visualizing Search History

The `search_data` attribute contains the complete evaluation history as a list of dictionaries. We can visualize where the optimizer explored.

In [ ]:
df = pd.DataFrame(func.search_data)
df["eval_num"] = range(len(df))

fig = plot_contour(RastriginFunction(n_dim=2), resolution=60)

In [ ]:
# Add search points
fig.add_trace(
    go.Scatter(
        x=df["x0"],
        y=df["x1"],
        mode="markers",
        marker=dict(
            size=8,
            color=df["eval_num"],
            colorscale="Reds",
            showscale=True,
            colorbar=dict(title="Eval #"),
        ),
        name="Search Points",
    )
)

# Mark best found
fig.add_trace(
    go.Scatter(
        x=[func.best_params["x0"]],
        y=[func.best_params["x1"]],
        mode="markers",
        marker=dict(size=15, color="lime", symbol="star"),
        name="Best Found",
    )
)

# Mark true optimum
fig.add_trace(
    go.Scatter(
        x=[0],
        y=[0],
        mode="markers",
        marker=dict(size=15, color="white", symbol="x"),
        name="True Optimum",
    )
)

fig.update_layout(title="Random Search on Rastrigin", width=650, height=550)
fig.show()

---

## 5. Search Space

Every function provides a `search_space` property: a dictionary mapping parameter names to arrays of valid values. This integrates directly with optimization libraries.

In [ ]:
func = AckleyFunction()

for name, values in func.search_space.items():
    print(f"{name}: {len(values)} values in [{values.min():.1f}, {values.max():.1f}]")

Higher dimensions work the same way:

In [ ]:
func_10d = SphereFunction(n_dim=10)
print(f"Parameters: {list(func_10d.search_space.keys())}")

---

## 6. Memory Caching

Many optimizers evaluate the same point multiple times. Enable `memory=True` to cache results and avoid redundant computation.

In [ ]:
# Simulate expensive evaluation with 10ms delay
func_slow = SphereFunction(n_dim=2, sleep=0.01)
func_cached = SphereFunction(n_dim=2, sleep=0.01, memory=True)

point = {"x0": 1.0, "x1": 2.0}

In [ ]:
# Without cache: 10 evaluations
start = time.time()
for _ in range(10):
    func_slow(point)
time_slow = time.time() - start

# With cache: 1 evaluation + 9 cache hits
start = time.time()
for _ in range(10):
    func_cached(point)
time_cached = time.time() - start

print(f"Without cache: {time_slow:.3f}s")
print(f"With cache:    {time_cached:.3f}s")
print(f"Speedup:       {time_slow/time_cached:.0f}x")

---

## 7. Comparing Optimizers

Here's a practical example: comparing two simple optimization strategies using Surfaces' automatic data collection.

In [ ]:
def random_search(func, n_iter=200):
    """Sample random points from the search space."""
    space = func.search_space
    for _ in range(n_iter):
        params = {k: np.random.choice(v) for k, v in space.items()}
        func(params)

In [ ]:
def hill_climb(func, n_iter=200, step=0.1):
    """Start random, then take small improving steps."""
    space = func.search_space
    current = {k: np.random.choice(v) for k, v in space.items()}
    current_score = func(current)

    for _ in range(n_iter - 1):
        candidate = {
            k: np.clip(current[k] + np.random.uniform(-step, step), space[k].min(), space[k].max())
            for k in space
        }
        score = func(candidate)
        if score < current_score:
            current, current_score = candidate, score

In [ ]:
# Run comparison
results = []
for FuncCls, name in [(SphereFunction, "Sphere"), (RastriginFunction, "Rastrigin")]:
    for opt, opt_name in [(random_search, "Random"), (hill_climb, "Hill Climb")]:
        np.random.seed(42)
        func = FuncCls(n_dim=2)
        opt(func)
        results.append(
            {
                "Function": name,
                "Optimizer": opt_name,
                "Best": func.best_score,
                "Optimum": func.f_global,
            }
        )

pd.DataFrame(results)

### Convergence Plot

Track how quickly each optimizer improves over time.

In [ ]:
fig = go.Figure()

for opt, name, color in [(random_search, "Random", "blue"), (hill_climb, "Hill Climb", "red")]:
    np.random.seed(42)
    func = RastriginFunction(n_dim=2)
    opt(func)

    scores = [d["score"] for d in func.search_data]
    best_so_far = np.minimum.accumulate(scores)
    fig.add_trace(go.Scatter(y=best_so_far, mode="lines", name=name, line=dict(color=color)))

fig.add_hline(y=0, line_dash="dash", line_color="green", annotation_text="Optimum")
fig.update_layout(
    title="Convergence on Rastrigin",
    xaxis_title="Evaluations",
    yaxis_title="Best Score",
    yaxis_type="log",
    width=650,
    height=400,
)
fig.show()

---

## 8. Pre-defined Suites

Surfaces includes curated function collections for standardized benchmarking.

In [ ]:
print("Available presets:")
print(f"  quick:    {len(presets.suites.quick):2d} functions - smoke tests")
print(f"  standard: {len(presets.suites.standard):2d} functions - academic benchmarks")
print(f"  bbob:     {len(presets.suites.bbob):2d} functions - COCO/BBOB competition")
print(f"  cec2014:  {len(presets.suites.cec2014):2d} functions - CEC 2014 competition")

In [ ]:
print("\nQuick suite:")
for cls in presets.suites.quick:
    print(f"  - {cls.__name__}")

---

## Bonus: The Eggholder

One of the most challenging test functions, with a highly irregular landscape.

In [ ]:
fig = plot_surface(EggholderFunction(), resolution=100)
fig.update_layout(title="Eggholder Function")
fig.show()

---

## Summary

**Key takeaways:**

| Feature | What it does |
|---------|-------------|
| 50+ functions | Diverse optimization challenges |
| Simple API | Callable with dict, array, list, or kwargs |
| Auto tracking | `search_data`, `best_score`, `best_params` |
| Memory cache | Skip redundant evaluations |
| Visualization | Understand landscapes before optimizing |
| Presets | Academic and competition benchmarks |

**Next steps:**
- Explore all functions: `from surfaces.test_functions import *`
- Try ML hyperparameter functions (require scikit-learn)
- Check engineering design problems with constraints
- Add noise to test robustness: `SphereFunction(noise=GaussianNoise(sigma=0.1))`